In [12]:
# import libraries
import os 
import csv
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from PIL import Image
import glob
from PIL import ImageFile
from sklearn.cluster import KMeans
import transformers
from sentence_transformers import SentenceTransformer
from PIL import Image, UnidentifiedImageError
import sklearn
ImageFile.LOAD_TRUNCATED_IMAGES = True
import re
import random
from spellchecker import SpellChecker
import inflect

# don't show the warning
import warnings
warnings.filterwarnings("ignore")

In [13]:
df = pd.read_csv('final_data_exploration.csv')

In [14]:
# applying clustering to the data from 


model = SentenceTransformer('all-mpnet-base-v2')

label_embeddings = model.encode(df['label'].tolist())

max_clusters = 10
all_silhouette_scores = []
for num_clusters in range(2, max_clusters + 1) :
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init='auto')
    cluster_labels = kmeans.fit_predict(label_embeddings)
    silhouette_sc = sklearn.metrics.silhouette_score(label_embeddings, cluster_labels)
    all_silhouette_scores.append(silhouette_sc)

optimal_clusters = all_silhouette_scores.index(max(all_silhouette_scores)) + 2

kmeans = KMeans(n_clusters=optimal_clusters, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(label_embeddings)

label_to_cluster = {label: cluster for label, cluster in zip(df['label'].unique(), cluster_labels)}
df[f'optimal_clusters_{optimal_clusters}'] = df['label'].map(label_to_cluster)


In [23]:
# Get cluster-wise embeddings
cluster_embeddings = []
for cluster in df['optimal_clusters_10'].unique():
    cluster_data = df[df['optimal_clusters_10'] == cluster]
    cluster_embeddings.append(model.encode(cluster_data['label'].tolist()))

# Find optimal number of clusters for each cluster
max_clusters = 30
silhouette_scores = []
for cluster_emb in cluster_embeddings:
    cluster_silhouette_scores = []
    for num_clusters in range(2, max_clusters + 1):
        kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init='auto')
        cluster_labels = kmeans.fit_predict(cluster_emb)
        cluster_silhouette_sc = sklearn.metrics.silhouette_score(cluster_emb, cluster_labels)
        cluster_silhouette_scores.append(cluster_silhouette_sc)
    silhouette_scores.append(cluster_silhouette_scores)

# Determine optimal number of clusters for each cluster
optimal_clusters = [scores.index(max(scores)) + 2 for scores in silhouette_scores]
print(f"Optimal number of clusters for each cluster: {optimal_clusters}")

# Perform final clustering and assign labels
df['optimal_clusters_10_30'] = 0
for i, cluster in enumerate(df['optimal_clusters_10'].unique()):
    cluster_data = df[df['optimal_clusters_10'] == cluster]
    kmeans = KMeans(n_clusters=optimal_clusters[i], random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(model.encode(cluster_data['label'].tolist()))
    cluster_label_to_id = {label: id for label, id in zip(cluster_data['label'].unique(), cluster_labels)}
    df.loc[df['optimal_clusters_10'] == cluster, 'optimal_clusters_10_30'] = df.loc[df['optimal_clusters_10'] == cluster, 'label'].map(cluster_label_to_id)

df[['label', 'optimal_clusters_10', 'optimal_clusters_10_30']].head(10)

Optimal number of clusters for each cluster: [30, 30, 30, 30, 30, 30, 30, 30, 30, 30]


,label,optimal_clusters_10,optimal_clusters_10_30
0,railway grinding machine,3,5
1,rail grinder,3,5
2,rail grinder with worker,3,5
3,railway grinding machine with worker,3,25
4,worker,8,12
5,railgrinder of the united state,4,24
6,ceramic mold,8,14
7,casting mound,8,17
8,feeler gage,4,29
9,feeler gauge,4,29


In [24]:
for score in silhouette_scores:
    print(max(score))

0.38279307
0.4448088
0.19634655
0.3975303
0.7649762
0.48485628
0.26440796
0.5816861
0.22289751
0.46539646


In [ ]:
# cluster_zero = df[df['optimal_clusters_10'] == 0]
# cluster_one = df[df['optimal_clusters_10'] == 1]
# cluster_two = df[df['optimal_clusters_10'] == 2]
# cluster_three = df[df['optimal_clusters_10'] == 3]
# cluster_four = df[df['optimal_clusters_10'] == 4]
# cluster_five = df[df['optimal_clusters_10'] == 5]
# cluster_six = df[df['optimal_clusters_10'] == 6]
# cluster_seven = df[df['optimal_clusters_10'] == 7]
# cluster_eight = df[df['optimal_clusters_10'] == 8]
# cluster_nine = df[df['optimal_clusters_10'] == 9]

# zero_label_embeddings = model.encode(cluster_zero['label'].tolist())
# one_label_embeddings = model.encode(cluster_one['label'].tolist())
# two_label_embeddings = model.encode(cluster_two['label'].tolist())
# three_label_embeddings = model.encode(cluster_three['label'].tolist())
# four_label_embeddings = model.encode(cluster_four['label'].tolist())
# five_label_embeddings = model.encode(cluster_five['label'].tolist())
# six_label_embeddings = model.encode(cluster_six['label'].tolist())
# seven_label_embeddings = model.encode(cluster_seven['label'].tolist())
# eight_label_embeddings = model.encode(cluster_eight['label'].tolist())
# nine_label_embeddings = model.encode(cluster_nine['label'].tolist())

# max_clusters = 10
# zero_silhouette_scores = []
# one_silhouette_scores = []
# two_silhouette_scores = []
# three_silhouette_scores = []
# four_silhouette_scores = []
# five_silhouette_scores = []
# six_silhouette_scores = []
# seven_silhouette_scores = []
# eight_silhouette_scores = []
# nine_silhouette_scores = []

# for num_clusters in range(2, max_clusters + 1) :
#     zero_kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init='auto')
#     zero_cluster_labels = zero_kmeans.fit_predict(zero_label_embeddings)
#     zero_silhouette_sc = sklearn.metrics.silhouette_score(zero_label_embeddings, zero_cluster_labels)
#     zero_silhouette_scores.append(silhouette_sc)

#     one_kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init='auto')
#     one_cluster_labels = one_kmeans.fit_predict(one_label_embeddings)
#     one_silhouette_sc = sklearn.metrics.silhouette_score(one_label_embeddings, one_cluster_labels)
#     one_silhouette_scores.append(silhouette_sc)

#     two_kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init='auto')
#     two_cluster_labels = two_kmeans.fit_predict(two_label_embeddings)
#     two_silhouette_sc = sklearn.metrics.silhouette_score(two_label_embeddings, two_cluster_labels)
#     two_silhouette_scores.append(silhouette_sc)

#     three_kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init='auto')
#     three_cluster_labels = three_kmeans.fit_predict(three_label_embeddings)
#     three_silhouette_sc = sklearn.metrics.silhouette_score(three_label_embeddings, three_cluster_labels)
#     three_silhouette_scores.append(silhouette_sc)

#     four_kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init='auto')
#     four_cluster_labels = four_kmeans.fit_predict(four_label_embeddings)
#     four_silhouette_sc = sklearn.metrics.silhouette_score(four_label_embeddings, four_cluster_labels)
#     four_silhouette_scores.append(silhouette_sc)

#     five_kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init='auto')
#     five_cluster_labels = five_kmeans.fit_predict(five_label_embeddings)
#     five_silhouette_sc = sklearn.metrics.silhouette_score(five_label_embeddings, five_cluster_labels)
#     five_silhouette_scores.append(silhouette_sc)

#     six_kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init='auto')
#     six_cluster_labels = six_kmeans.fit_predict(six_label_embeddings)
#     six_silhouette_sc = sklearn.metrics.silhouette_score(six_label_embeddings, six_cluster_labels)
#     six_silhouette_scores.append(silhouette_sc)
    
#     seven_kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init='auto')
#     seven_cluster_labels = seven_kmeans.fit_predict(seven_label_embeddings)
#     seven_silhouette_sc = sklearn.metrics.silhouette_score(seven_label_embeddings, seven_cluster_labels)
#     seven_silhouette_scores.append(silhouette_sc)

#     eight_kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init='auto')
#     eight_cluster_labels = eight_kmeans.fit_predict(eight_label_embeddings)
#     eight_silhouette_sc = sklearn.metrics.silhouette_score(eight_label_embeddings, eight_cluster_labels)
#     eight_silhouette_scores.append(silhouette_sc)

#     nine_kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init='auto')
#     nine_cluster_labels = nine_kmeans.fit_predict(nine_label_embeddings)
#     nine_silhouette_sc = sklearn.metrics.silhouette_score(nine_label_embeddings, nine_cluster_labels)
#     nine_silhouette_scores.append(silhouette_sc)

# zero_optimal_clusters = zero_silhouette_scores.index(max(zero_silhouette_scores)) + 2
# one_optimal_clusters = one_silhouette_scores.index(max(one_silhouette_scores)) + 2
# two_optimal_clusters = two_silhouette_scores.index(max(two_silhouette_scores)) + 2
# three_optimal_clusters = three_silhouette_scores.index(max(three_silhouette_scores)) + 2
# four_optimal_clusters = four_silhouette_scores.index(max(four_silhouette_scores)) + 2
# five_optimal_clusters = five_silhouette_scores.index(max(five_silhouette_scores)) + 2
# six_optimal_clusters = six_silhouette_scores.index(max(six_silhouette_scores)) + 2
# seven_optimal_clusters = seven_silhouette_scores.index(max(seven_silhouette_scores)) + 2
# eight_optimal_clusters = eight_silhouette_scores.index(max(eight_silhouette_scores)) + 2
# nine_optimal_clusters = nine_silhouette_scores.index(max(nine_silhouette_scores)) + 2
    

# zero_kmeans = KMeans(n_clusters=zero_optimal_clusters, random_state=42, n_init=10)
# zero_cluster_labels = zero_kmeans.fit_predict(zero_label_embeddings)
# zero_label_to_cluster = {label: cluster for label, cluster in zip(cluster_zero['label'].unique(), zero_cluster_labels)}
# cluster_zero['optimal_clusters'] = cluster_zero['label'].map(zero_label_to_cluster)

# one_kmeans = KMeans(n_clusters=one_optimal_clusters, random_state=42, n_init=10)
# one_cluster_labels = one_kmeans.fit_predict(one_label_embeddings)
# one_label_to_cluster = {label: cluster for label, cluster in zip(cluster_one['label'].unique(), one_cluster_labels)}
# cluster_one['optimal_clusters'] = cluster_one['label'].map(one_label_to_cluster)

# two_kmeans = KMeans(n_clusters=two_optimal_clusters, random_state=42, n_init=10)
# two_cluster_labels = two_kmeans.fit_predict(two_label_embeddings)
# two_label_to_cluster = {label: cluster for label, cluster in zip(cluster_two['label'].unique(), two_cluster_labels)}
# cluster_two['optimal_clusters'] = cluster_two['label'].map(two_label_to_cluster)

# three_kmeans = KMeans(n_clusters=three_optimal_clusters, random_state=42, n_init=10)
# three_cluster_labels = three_kmeans.fit_predict(three_label_embeddings)
# three_label_to_cluster = {label: cluster for label, cluster in zip(cluster_three['label'].unique(), three_cluster_labels)}
# cluster_three['optimal_clusters'] = cluster_three['label'].map(three_label_to_cluster)

# four_kmeans = KMeans(n_clusters=four_optimal_clusters, random_state=42, n_init=10)
# four_cluster_labels = four_kmeans.fit_predict(four_label_embeddings)
# four_label_to_cluster = {label: cluster for label, cluster in zip(cluster_four['label'].unique(), four_cluster_labels)}
# cluster_four['optimal_clusters'] = cluster_four['label'].map(four_label_to_cluster)

# five_kmeans = KMeans(n_clusters=five_optimal_clusters, random_state=42, n_init=10)
# five_cluster_labels = five_kmeans.fit_predict(five_label_embeddings)
# five_label_to_cluster = {label: cluster for label, cluster in zip(cluster_five['label'].unique(), five_cluster_labels)}
# cluster_five['optimal_clusters'] = cluster_five['label'].map(five_label_to_cluster)

# six_kmeans = KMeans(n_clusters=six_optimal_clusters, random_state=42, n_init=10)
# six_cluster_labels = six_kmeans.fit_predict(six_label_embeddings)
# six_label_to_cluster = {label: cluster for label, cluster in zip(cluster_six['label'].unique(), six_cluster_labels)}
# cluster_six['optimal_clusters'] = cluster_six['label'].map(six_label_to_cluster)

# seven_kmeans = KMeans(n_clusters=seven_optimal_clusters, random_state=42, n_init=10)
# seven_cluster_labels = seven_kmeans.fit_predict(seven_label_embeddings)
# seven_label_to_cluster = {label: cluster for label, cluster in zip(cluster_seven['label'].unique(), seven_cluster_labels)}
# cluster_seven['optimal_clusters'] = cluster_seven['label'].map(seven_label_to_cluster)

# eight_kmeans = KMeans(n_clusters=eight_optimal_clusters, random_state=42, n_init=10)
# eight_cluster_labels = eight_kmeans.fit_predict(eight_label_embeddings)
# eight_label_to_cluster = {label: cluster for label, cluster in zip(cluster_eight['label'].unique(), eight_cluster_labels)}
# cluster_eight['optimal_clusters'] = cluster_eight['label'].map(eight_label_to_cluster)

# nine_kmeans = KMeans(n_clusters=nine_optimal_clusters, random_state=42, n_init=10)
# nine_cluster_labels = nine_kmeans.fit_predict(nine_label_embeddings)
# nine_label_to_cluster = {label: cluster for label, cluster in zip(cluster_nine['label'].unique(), nine_cluster_labels)}
# cluster_nine['optimal_clusters'] = cluster_nine['label'].map(nine_label_to_cluster)


    

# print(f"good clusters in cluster zero: {zero_optimal_clusters}")
# print(f"good clusters in cluster one: {one_optimal_clusters}")
# print(f"good clusters in cluster two: {two_optimal_clusters}")
# print(f"good clusters in cluster three: {three_optimal_clusters}")
# print(f"good clusters in cluster four: {four_optimal_clusters}")
# print(f"good clusters in cluster five: {five_optimal_clusters}")
# print(f"good clusters in cluster six: {six_optimal_clusters}")
# print(f"good clusters in cluster seven: {seven_optimal_clusters}")
# print(f"good clusters in cluster eight: {eight_optimal_clusters}")
# print(f"good clusters in cluster nine: {nine_optimal_clusters}")


good clusters in cluster zero: 2
good clusters in cluster one: 2
good clusters in cluster two: 2
good clusters in cluster three: 2
good clusters in cluster four: 2
good clusters in cluster five: 2
good clusters in cluster six: 2
good clusters in cluster seven: 2
good clusters in cluster eight: 2
good clusters in cluster nine: 2
